<a href="https://colab.research.google.com/github/jvance7-ut/Install4/blob/main/scraper1_cleaner1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Please note that it is not legal to scrape all sites.  Please check current standards for the site you intend to scrape.

BeautifulSoup Scraper tutorial

https://finance.yahoo.com/markets/stocks/most-active/



**Import packages**

In [1]:
#install packages if needed
#!pip install numpy==1.19.5
#!pip install beautifulsoup4==4.6.3


In [2]:
from bs4 import BeautifulSoup #scraping
import requests #Open URL
import pandas as pd #dataframe

**Scrape the data**

In [3]:
#making a GET request
r = requests.get('https://finance.yahoo.com/markets/stocks/most-active/')

#check status code for response received
#success code - 200
print(r)


<Response [200]>


In [4]:
#parsing the HTML
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="desktop neo-green dock-upscale failsafe" data-color-scheme="light" lang="en-US" theme="light">
 <head>
  <meta charset="utf-8"/>
  <meta content="guce.yahoo.com" name="oath:guce:consent-host"/>
  <link crossorigin="anonymous" href="//s.yimg.com" rel="preconnect"/>
  <link href="//geo.yahoo.com" rel="preconnect"/>
  <link href="//query1.finance.yahoo.com" rel="preconnect"/>
  <link href="//query2.finance.yahoo.com" rel="preconnect"/>
  <link href="//consent.cmp.oath.com" rel="preconnect"/>
  <link as="worker" href="/__rapidworker-1.2.js" rel="preload"/>
  <link href="https://s.yimg.com/uc/finance/webcore/js/_staticFinProtobuf.4b1559b8e4645fd93a12.js" rel="preconnect"/>
  <link href="https://cdn.jsdelivr.net/npm/protobufjs@7.1.2/dist/minimal/protobuf.min.js" rel="preconnect"/>
  <script>
   window.finNeoPageStart = Date.now();
  </script>
  <script>
   (function(){if(!window.YAHOO){window.YAHOO={}}; window.YAHOO.context={"consent":{"allowContentPersonalizatio

In [5]:
#we want the table values
#the HTML value for our table is class_= yf-paf8n5
#if you need help finding what values you want:
#1. go to the website
#2. right click and select 'Inspect' from the menu
#3. hover over the HTML until ONLY THE AREA YOU WANT is blue
#4. use the HTML tags from that line

table_names = soup.find('table', class_='yf-paf8n5')

# Extract data from the tbody
if table_names:
    # Loop through rows
    for row in table_names.find_all('th'):
      #column names use 'th'
        print(row.text)
    #print(len(row.text))

else:
    print("raw_table not found.")


data_names = []
for row in table_names.find_all('th'):
    data_names.append(row.text.strip())


print("   ")
print(data_names)

print(len(data_names))

Symbol  
Name  
  
Price  
Change  
Change %  
Volume  
Avg Vol (3M)  
Market Cap  
P/E Ratio (TTM)  
52 Wk Change %  
52 Wk Range  
   
['Symbol', 'Name', '', 'Price', 'Change', 'Change %', 'Volume', 'Avg Vol (3M)', 'Market Cap', 'P/E Ratio (TTM)', '52 Wk Change %', '52 Wk Range']
12


In [6]:
# Find the table cell values with the specific class using tbody
tbody = soup.find('tbody', class_='body yf-paf8n5')

# Extract data from the tbody
if tbody:
    # Loop through rows
    for row in tbody.find_all('tr'):
        # Loop through cells
        for cell in row.find_all('td'):
            print(cell.text)
    #cell values use 'tr' and 'td'
else:
    print("Tbody not found.")

  NVDA    
NVIDIA Corporation  
   
 124.83 +6.18 (+5.21%) 
 +6.18 
 +5.21% 
 259.005M 
243.476M 
 3.057T 
49.34 
70.37% 
 
  RGTI    
Rigetti Computing, Inc.  
   
 13.83 +0.11 (+0.80%) 
 +0.11 
 +0.80% 
 117.095M 
147.537M 
 3.873B 
- 
1,082.76% 
 
  AMD    
Advanced Micro Devices, Inc.  
   
 112.01 -7.49 (-6.27%) 
 -7.49 
 -6.27% 
 109.88M 
36.966M 
 181.771B 
98.25 
-29.44% 
 
  F    
Ford Motor Company  
   
 10.01 -0.15 (-1.48%) 
 -0.15 
 -1.48% 
 117.804M 
60.964M 
 39.783B 
11.38 
-20.81% 
 
  PLTR    
Palantir Technologies Inc.  
   
 101.36 -2.47 (-2.38%) 
 -2.47 
 -2.38% 
 90.308M 
84.646M 
 230.9B 
533.47 
323.62% 
 
  SMCI    
Super Micro Computer, Inc.  
   
 31.49 +2.33 (+7.99%) 
 +2.33 
 +7.99% 
 81.534M 
68.282M 
 18.439B 
15.67 
-58.22% 
 
  SNAP    
Snap Inc.  
   
 10.63 -0.97 (-8.36%) 
 -0.97 
 -8.36% 
 80.226M 
29.121M 
 18.032B 
- 
4.50% 
 
  UBER    
Uber Technologies, Inc.  
   
 64.48 -5.27 (-7.56%) 
 -5.27 
 -7.56% 
 74.773M 
25.596M 
 135.776B 
14.14 
-2.60

**Turn the data into a dataframe using Pandas**

In [7]:
data = []
#empty data frame
for row in tbody.find_all('tr'):
    row_data = []
    for cell in row.find_all('td'):
        row_data.append(cell.text.strip())
    data.append(row_data)
#fill in empty dataframe row by row and column by column

#create a temporary dataframe for cleaning (optional)
df = pd.DataFrame(data)

##Our data added the values from column 4 and 5 to column 3
## we need to remove the extra information
# Function to remove everything after the first space
def remove_after_space(text):
    if isinstance(text, str):
        return text.split(' ')[0]
    return text
# Apply the function to the column
df[3] = df[3].apply(remove_after_space)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,NVDA,NVIDIA Corporation,,124.83,+6.18,+5.21%,259.005M,243.476M,3.057T,49.34,70.37%,
1,RGTI,"Rigetti Computing, Inc.",,13.83,+0.11,+0.80%,117.095M,147.537M,3.873B,-,"1,082.76%",
2,AMD,"Advanced Micro Devices, Inc.",,112.01,-7.49,-6.27%,109.88M,36.966M,181.771B,98.25,-29.44%,
3,F,Ford Motor Company,,10.01,-0.15,-1.48%,117.804M,60.964M,39.783B,11.38,-20.81%,
4,PLTR,Palantir Technologies Inc.,,101.36,-2.47,-2.38%,90.308M,84.646M,230.9B,533.47,323.62%,
5,SMCI,"Super Micro Computer, Inc.",,31.49,+2.33,+7.99%,81.534M,68.282M,18.439B,15.67,-58.22%,
6,SNAP,Snap Inc.,,10.63,-0.97,-8.36%,80.226M,29.121M,18.032B,-,4.50%,
7,UBER,"Uber Technologies, Inc.",,64.48,-5.27,-7.56%,74.773M,25.596M,135.776B,14.14,-2.60%,
8,LCID,"Lucid Group, Inc.",,2.9100,-0.0200,-0.68%,67.972M,89.12M,8.764B,-,-16.52%,
9,GOOGL,Alphabet Inc.,,191.33,-15.05,-7.29%,69.856M,27.838M,2.343T,23.80,41.44%,


In [8]:
#assign the data_names as the column names
df.columns = data_names

# Drop the blank columns (had graphs on website)
df.drop(df.columns[2], axis=1, inplace=True)
df.drop(df.columns[10], axis=1, inplace=True)


df

,Symbol,Name,Price,Change,Change %,Volume,Avg Vol (3M),Market Cap,P/E Ratio (TTM),52 Wk Change %
0,NVDA,NVIDIA Corporation,124.83,+6.18,+5.21%,259.005M,243.476M,3.057T,49.34,70.37%
1,RGTI,"Rigetti Computing, Inc.",13.83,+0.11,+0.80%,117.095M,147.537M,3.873B,-,"1,082.76%"
2,AMD,"Advanced Micro Devices, Inc.",112.01,-7.49,-6.27%,109.88M,36.966M,181.771B,98.25,-29.44%
3,F,Ford Motor Company,10.01,-0.15,-1.48%,117.804M,60.964M,39.783B,11.38,-20.81%
4,PLTR,Palantir Technologies Inc.,101.36,-2.47,-2.38%,90.308M,84.646M,230.9B,533.47,323.62%
5,SMCI,"Super Micro Computer, Inc.",31.49,+2.33,+7.99%,81.534M,68.282M,18.439B,15.67,-58.22%
6,SNAP,Snap Inc.,10.63,-0.97,-8.36%,80.226M,29.121M,18.032B,-,4.50%
7,UBER,"Uber Technologies, Inc.",64.48,-5.27,-7.56%,74.773M,25.596M,135.776B,14.14,-2.60%
8,LCID,"Lucid Group, Inc.",2.9100,-0.0200,-0.68%,67.972M,89.12M,8.764B,-,-16.52%
9,GOOGL,Alphabet Inc.,191.33,-15.05,-7.29%,69.856M,27.838M,2.343T,23.80,41.44%


**Clean the data**

In [9]:
#### column by column cleaning

##Price
df['Price'] = pd.to_numeric(df['Price'])

##Change
df['Change'] = df['Change'].str.replace('+', '', regex=False)
#remove '+' but keep '-' to show negative valuse
df['Change'] = pd.to_numeric(df['Change'])
#make column values numeric ('-' will be read as negative here)

##Change %
df['Change %'] = df['Change %'].str.replace('+', '', regex=False)
df['Change %'] = df['Change %'].str.replace('%', '', regex=False)
#remove '+' and '%' signs
df['Change %'] = pd.to_numeric(df['Change %'])
#change column to numeric
df['Change %'] = df['Change %']*0.01
#put percent in decimal format for possible calculations

###M/B/T values into numeric
##Volume
df['Volume'] = df['Volume'].str.replace('.', '', regex=False)
#get rid of the decimal for ease of computation
df['Volume'] = df['Volume'].str.replace('M', '000', regex=False)
df['Volume'] = df['Volume'].str.replace('B', '000000', regex=False)
df['Volume'] = df['Volume'].str.replace('T', '000000000', regex=False)
#add the appropriate number of 0s
df['Volume'] = pd.to_numeric(df['Volume'])
#change to numeric

##Avg Vol (3M)	(Average volume over 3 months)
df['Avg Vol (3M)'] = df['Avg Vol (3M)'].str.replace('.', '', regex=False)
#get rid of the decimal for ease of computation
df['Avg Vol (3M)'] = df['Avg Vol (3M)'].str.replace('M', '000', regex=False)
df['Avg Vol (3M)'] = df['Avg Vol (3M)'].str.replace('B', '000000', regex=False)
df['Avg Vol (3M)'] = df['Avg Vol (3M)'].str.replace('T', '000000000', regex=False)
#add the appropriate number of 0s
df['Avg Vol (3M)'] = pd.to_numeric(df['Avg Vol (3M)'])
#change to numeric

##Market Cap
df['Market Cap'] = df['Market Cap'].str.replace('.', '', regex=False)
#get rid of the decimal for ease of computation
df['Market Cap'] = df['Market Cap'].str.replace('M', '000', regex=False)
df['Market Cap'] = df['Market Cap'].str.replace('B', '000000', regex=False)
df['Market Cap'] = df['Market Cap'].str.replace('T', '000000000', regex=False)
#add the appropriate number of 0s
df['Market Cap'] = pd.to_numeric(df['Market Cap'])
#change to numeric

##P/E Ratio (TTM)
df['P/E Ratio (TTM)'] = df['P/E Ratio (TTM)'].replace('-','0.0')
#in this case, the '-' are read literally, not as a null value, so they need to
#be replaced manually.  These rows is either a zero or negative value, so they do
#not fill it in.  Remember this when using column for computations.
df['P/E Ratio (TTM)'] = pd.to_numeric(df['P/E Ratio (TTM)'])
#change column to numeric
df['P/E Ratio (TTM)']

##52 Wk Change %
df['52 Wk Change %'] = df['52 Wk Change %'].str.replace('%', '', regex=False)
df['52 Wk Change %'] = df['52 Wk Change %'].str.replace(',', '', regex=False)
#remove '%' and ',' to allow type to be changed to numeric
df['52 Wk Change %'] = pd.to_numeric(df['52 Wk Change %'])
#make column numeric
df['52 Wk Change %'] = df['52 Wk Change %']*0.01
#put percent value in decimal format


In [10]:
##check your work
#numeric_columns = df.select_dtypes(include='number')
#numeric_columns

In [11]:
#make a permanent dataframe name
most_active_stocks = df
most_active_stocks

   Symbol                          Name   Price   Change  Change %     Volume  \
0    NVDA            NVIDIA Corporation  124.83   6.1800    0.0521  259005000   
1    RGTI       Rigetti Computing, Inc.   13.83   0.1100    0.0080  117095000   
2     AMD  Advanced Micro Devices, Inc.  112.01  -7.4900   -0.0627   10988000   
3       F            Ford Motor Company   10.01  -0.1500   -0.0148  117804000   
4    PLTR    Palantir Technologies Inc.  101.36  -2.4700   -0.0238   90308000   
5    SMCI    Super Micro Computer, Inc.   31.49   2.3300    0.0799   81534000   
6    SNAP                     Snap Inc.   10.63  -0.9700   -0.0836   80226000   
7    UBER       Uber Technologies, Inc.   64.48  -5.2700   -0.0756   74773000   
8    LCID             Lucid Group, Inc.    2.91  -0.0200   -0.0068   67972000   
9   GOOGL                 Alphabet Inc.  191.33 -15.0500   -0.0729   69856000   
10   INTC             Intel Corporation   19.65   0.3600    0.0187   61096000   
11   GRAB         Grab Holdi

**Keep a copy of your dataset**

In [12]:
###most Python

#from os import path
#print(path)
#most_active_stocks.to_csv('most_active_stocks.csv', index=False)


In [13]:
###Colab notebook

##if you want to save it to your google drive
#from google.colab import drive
#drive.mount('/content/drive')
#allow permissions

most_active_stocks.to_csv('most_active_stocks.csv', index=False)
#create CSV

from google.colab import files
files.download('most_active_stocks.csv')
#download CSV

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>